In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mbipy
import fabio
import fabio.edfimage as edf
import fabio.tifimage as tif
import numpy as np
import os
import glob
from scipy.ndimage import uniform_filter,generic_filter,uniform_filter1d

# Fonction pour calculer l'écart type dans une direction donnée avec une fenêtre glissante
def calculate_std_dev(image_array, direction, window_size):
    rows, cols = image_array.shape
    std_dev_map = np.zeros_like(image_array, dtype=float)
    half_window = window_size // 2

    for i in range(rows):
        for j in range(cols):
            values = []
            x, y = i, j
            while 0 <= x < rows and 0 <= y < cols and len(values) < window_size:
                values.append(image_array[x, y])
                x += direction[0]
                y += direction[1]
            std_dev_map[i, j] = np.std(values) if len(values) > 1 else 0

    return std_dev_map


import fabio.tifimage as tif
def open_image(filename):
    """opens a 2D image

    Args:
        filename (str): file name

    Returns:
        (numpy.ndarray): 2D image
    """
    filename = str(filename)
    if "edf" in filename:
        im = fabio.open(filename)
        return im.data
    elif "tif" in filename or "tiff" in filename:
        im = fabio.open(filename)
        return im.data


def create_list_of_files(folder_name, extension):
    """creating a list of files with a corresponding extension in an input folder

    Args:
        folder_name (str): folder name
        extension (str):   extension of the target files

    Returns:
        (str): the list of files sorted
    """
    if not os.path.exists(folder_name):
        raise Exception('Error: Given path does not exist.')
    list_of_files = glob.glob(folder_name + '/*' + extension)
    list_of_files.sort()
    if len(list_of_files) == 0:
        if "tif" in extension.lower():
            extension = "edf"
            list_of_files = glob.glob(folder_name + '/*' + extension)
            list_of_files.sort()
        elif "edf" in extension.lower():
            extension = "tif"
            list_of_files = glob.glob(folder_name + '/*' + extension)
            list_of_files.sort()
        else:
            raise Exception('Error: No file corresponds to the given extension: .' + extension)

    if len(list_of_files) == 0:
        raise Exception('Error: No file corresponds to the following extensions: .tif and .edf')
    return list_of_files       

def open_sequence(filenames_or_input_folder, extension="tif"):
    """opens a sequence of images

    Args:
        filenames_or_input_folder (str): file names
        extension (str):                 files extension

    Returns:
        (numpy.ndarray): sequence of 2D images
    """
    # If the given arg is empty, we raise an error
    if len(filenames_or_input_folder) == 0:
        raise Exception('Error: no file corresponds to the given path/extension')
    # We check if the given filenames is a regular expression of input files:
    if type(filenames_or_input_folder) != list:
        # We try opening .extension files
        list_of_files = create_list_of_files(filenames_or_input_folder, extension)
    else:
        list_of_files = filenames_or_input_folder
    # If the created list_of_files is empty
    if len(list_of_files) == 0:
        raise Exception('Error: no file corresponds to the given path/extension')

    # Next line is computed iff given regex/list of files correspond to existing files that can be opened
    if len(list_of_files) > 0:
        reference_image = open_image(str(list_of_files[0]))
        height, width, = reference_image.shape[-2:]
        # We create an empty image sequence
        sequence = np.zeros((len(list_of_files), height, width), dtype=np.float32)
        # We fill the created empty sequence
        for i, file in enumerate(list_of_files):
            image = open_image(str(file))
            sequence[i, :, :] = image
        return sequence

def save_tif_image(image, filename, bit=32, rgb=False, header=None):
    """saves an image to .tif format (either int16 or float32)

    Args:
        image (numpy.ndarray): 2D image
        filename (str):        file name
        bit (int):             16: int16, 32: float32
        rgb (bool):            rgb format
        header (str):          header

    Returns:
        None
    """
    create_directory(remove_filename_in_path(filename))
    if rgb:
        imageio.imwrite(filename + '.tif', image)
    elif header:
        if bit == 32:
            tif.TifImage(data=image.astype(np.float32), header=header).write(filename + '.tif')
        else:
            tif.TifImage(data=image.astype(np.uint16), header=header).write(filename + '.tif')
    elif bit == 32:
        tif.TifImage(data=image.astype(np.float32)).write(filename + '.tif')
    else:
        tif.TifImage(data=image.astype(np.uint16)).write(filename + '.tif')


def save_tif_sequence(image, path, bit=32, header=None):
    """saves a sequence of images to .tif format (either int16 or float32)

    Args:
        image (numpy.ndarray): 2D image
        path (str):            complete path + regular expression of file names
        bit (int):             16: int16, 32: float32
        header (str):          header

    Returns:
        None
    """
    for i in range(image.shape[0]):
        image_path = path + '{:04d}'.format(i)
        save_tif_image(image[i, :, :], image_path, bit, header=header)

def create_directory(path):
    """creates a directory at the specified path

    Args:
        path (str): complete path

    Returns:
        None
    """
    if not os.path.exists(path):
        os.makedirs(path)


def remove_filename_in_path(path):
    """remove the file name from a path

    Args:
        path (str): complete path

    Returns:
        complete path without the file name
    """
    splitter = "\\" if len(path.split("\\")) > 1 else "/"
    path_list = path.split(splitter)[:-1]
    return "".join(elt + splitter for elt in path_list)


def remove_last_folder_in_path(path):
    """remove the last folder from a path

    Args:
        path (str): complete path

    Returns:
        complete path without the last folder
    """
    splitter = "\\" if len(path.split("\\")) > 1 else "/"

    path_list = path.split(splitter)[:-2]
    return "".join(elt + splitter for elt in path_list)

def window_stdev(X, window_size):
    if X.ndim<3: 
        c1 = uniform_filter(X, window_size, mode='reflect')
        c2 = uniform_filter(X*X, window_size, mode='reflect')
        result=np.sqrt(c2 - c1*c1)

    else : 
        nSlices=X.shape[0]
        result=np.zeros_like(X,dtype=np.float32)
        print('In 3D')
        for i in range(nSlices):
            c1 = uniform_filter(X[i,:,:], window_size, mode='reflect')
            c2 = uniform_filter(X[i,:,:]*X[i,:,:], window_size, mode='reflect')
            result[i,:,:]=np.sqrt(c2 - c1*c1)



    return result





In [2]:
import napari as napa


samples=open_sequence('/Users/embrun/Downloads/Distance_22000/Proj_0')
refs=open_sequence('/Users/embrun/Downloads/Distance_22000/Flat_0')

#samples=samples[0:5,:,:]
#refs=refs[0:5,:,:]

att=samples/refs



visu=napa.view_image(att)
visu.add_image(samples)
visu.add_image(refs )

Exception: Error: Given path does not exist.

In [3]:
transmission=samples/refs
#darkField=1.0-(1.0/transmission * window_stdev(singleSamp,30)/window_stdev(singleRef,30))
stdMapRef=window_stdev(samples,8)
stdMapSample=window_stdev(refs,8)
darkField=np.abs(1.0-(1.0/transmission * stdMapSample/stdMapRef))

medianDarkField=np.median(darkField,axis=0)
medianTransmission=np.median(transmission,axis=0)


#darkField=(stdMapRef- stdMapSample)/stdMapRef
#darkField=np.abs(darkField)
#darkField[darkField>1.]=1.
#darkField[darkField<0.]=0.
visu3=napa.view_image(att)

visu3.add_image(transmission)
visu3.add_image(darkField)
visu3.add_image(stdMapSample)
visu3.add_image(stdMapRef)
visu3.add_image(medianDarkField)
visu3.add_image(medianTransmission)





In 3D
In 3D


<Image layer 'medianTransmission' at 0x7fb8d4d26610>

In [24]:
save_tif_image(medianDarkField,'/Users/embrun/Downloads/Distance_22000/MedianDarkField.tif')

In [25]:

save_tif_image(medianTransmission,'/Users/embrun/Downloads/Distance_22000/MedianTransmission.tif')